## Basic functions

In [3]:
import tensorflow as tf  # tf 2.x
import tensornetwork as tn
tn.set_default_backend("tensorflow")
from tensornetwork import ncon
import matplotlib.pyplot as plt
import math
import cmath
import numpy as np

try:
    import QGOpt as qgo
except ImportError:
    !pip install git+https://github.com/LuchnikovI/QGOpt
    import QGOpt as qgo

In [4]:
def matr_Ry(angle):
    return tf.constant([[math.cos(angle/2), -math.sin(angle/2)],
                        [+math.sin(angle/2), math.cos(angle/2)]], dtype=tf.complex64)

def matr_Rx(angle):
    return tf.constant([[math.cos(angle/2) + 0j, -1j * math.sin(angle/2)],
                        [-1j * math.sin(angle/2), math.cos(angle/2) + 0j]], dtype=tf.complex64)

def matr_Rz(angle):
    return tf.constant([[math.cos(angle/2) -1j * math.sin(angle/2), 0],
                        [0, math.cos(angle/2) + 1j * math.sin(angle/2)]], dtype=tf.complex64)

Hadamard = tf.constant([[1, 1],
                        [1, -1]], dtype=tf.complex64) / math.sqrt(2)

In [5]:
@tf.function
def kron(A, B):
    """
    Returns Kronecker product of two square matrices.

    Args:
        A: complex valued tf tensor of shape (dim1, dim1)
        B: complex valued tf tensor of shape (dim2, dim2)

    Returns:
        complex valued tf tensor of shape (dim1 * dim2, dim1 * dim2),
        kronecker product of two matrices
    """

    dim1 = A.shape[-1]
    dim2 = B.shape[-1]
    AB = tf.transpose(tf.tensordot(A, B, axes=0), (0, 2, 1, 3))
    return tf.reshape(AB, (dim1 * dim2, dim1 * dim2))

In [252]:
class QuantumCircuits:

    def __init__(self, gates, n):
        self.gates = gates
        self.circuits = {}
        self.n = n
        self.in_states = n * [tf.constant([1, 0], dtype=tf.complex64)]
        
    def add_circuit(self, tn_template, name):
        self.circuits[name] = tn_template
        #template = [gates_order, ncon]
        
    def evaluate (self, indices, name): #indices.shape = (bs, n)
        out_tensors = tf.one_hot(indices, 2, dtype=tf.complex64) #(bs, n, 2); n - нумерует тензор
        out_tensors = [out_tensors[:, i, :] for i in range(self.n)]
        tensors, net_struc, con_order, out_order = self.circuits[name]
        tensors = out_tensors + self.in_states + [self.gates[i] for i in tensors]
        #print(net_struc)
        #print(len(tensors))
        for i in np.arange(len(net_struc)):
            for j in np.arange(len(net_struc[i])):
                if isinstance(net_struc[i][j], int):
                    if net_struc[i][j] < 0:
                        #print('CCC', net_struc[i][j])
                        net_struc[i][j] = 'out' + str(-net_struc[i][j])
        net_struc = [[-1, 'out' + str(i)] for i in range(1, self.n+1)] + [[i] for i in range(1, self.n+1)] + net_struc
        #print(len(net_struc))
        con_order = ['out' + str(i) for i in range(1, self.n+1)] + [i for i in range(1, self.n+1)] + con_order
        #print(len(out_tensors))
        #print(net_struc)
        #print(con_order)
        
        #print("TENSOR 101", tensors[101])
        #print("NET STRUCT 101", net_struc[101])
        
        return ncon(tensors, net_struc, con_order, (-1,))

In [259]:
H = tf.constant([[1, 1],
                 [1, -1]], dtype=tf.complex64) / math.sqrt(2)

CNOT = tf.constant([[[[1, 0],
                    [0, 1]], 
                    [[0, 0],
                    [0, 0]]],
                    [[[0, 0],
                    [0, 0]],
                     [[0, 1],
                     [1, 0]]]], dtype=tf.complex64)

tn_template = [[0] + 49 * [1],
               [[51, 1]] + [[50+i+1, -i, i+1, 50+i] for i in range(1,49)] + [[-50, -49, 50, 99]],
              [i for i in range (51, 100)],
              [-i for i in range (1, 51)]]  #-i for i in range (1, 51)]

In [260]:
qc = QuantumCircuits([H, CNOT], 50)

In [261]:
qc.add_circuit(tn_template, 'GHZ')

In [262]:
aaa = qc.evaluate(tf.concat([tf.zeros((1, 50), dtype=tf.int32), tf.ones((1, 50), dtype=tf.int32)], axis=0), 'GHZ')

In [263]:
aaa

<tf.Tensor: shape=(2,), dtype=complex64, numpy=array([0.7071068+0.j, 0.7071068+0.j], dtype=complex64)>